In [ ]:
!pip install -q openai google-api-python-client google-auth google-auth-oauthlib google-auth-httplib2 gspread oauth2client PyMuPDF dateparser

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 21.9 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload service_account.json, credentials.json, labor_law.pdf

Saving service_account.json to service_account.json
Saving labor_law.pdf to labor_law.pdf
Saving credentials.json to credentials.json


In [ ]:
import os, getpass
api_key = os.getenv("OPENROUTER_API_KEY") or getpass.getpass("Enter your OpenRouter API key: ")
os.environ["OPENROUTER_API_KEY"] = api_key

Enter your OpenRouter API key: ··········


In [ ]:
import fitz  # PyMuPDF

def load_pdf_text(path):
    doc = fitz.open(path)
    return "\n".join([page.get_text() for page in doc])

policy_text = load_pdf_text("labor_law.pdf")

In [ ]:
from openai import OpenAI

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=api_key,
)

system_prompt = f"""
You are a professional legal assistant operating strictly under Saudi Arabian labor law.
Your tasks include:
- Answering user inquiries clearly and professionally in English.
- Referring to official policy (provided below) when needed.
- Logging every interaction to Google Sheets.
- Notifying the legal team via email when a question is asked.
- Managing the legal calendar: viewing, creating, deleting events on the user's behalf.
- NEVER disclose or leak any confidential document or user info.

---
EMBEDDED POLICY DOCUMENT:
{policy_text[:12000]}
... (trimmed)
"""

In [ ]:
import pickle
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build

SCOPES = ['https://www.googleapis.com/auth/calendar']

def authenticate_calendar():
    creds = None
    if os.path.exists('token.pkl'):
        with open('token.pkl', 'rb') as token:
            creds = pickle.load(token)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            flow.redirect_uri = 'urn:ietf:wg:oauth:2.0:oob'  # 👈🏽 REQUIRED for Colab manual code flow

            auth_url, _ = flow.authorization_url(prompt='consent')
            print("Go to this URL:", auth_url)

            code = input("Paste the authorization code here: ")
            flow.fetch_token(code=code)
            creds = flow.credentials

        with open('token.pkl', 'wb') as token:
            pickle.dump(creds, token)

    return build('calendar', 'v3', credentials=creds)

calendar_service = authenticate_calendar()


Go to this URL: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=466723531815-338d9a7vslai0i706spoodpc4jh6pldg.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=iihWjhDoFpie10wQSLjjBfeQdgCZah&prompt=consent&access_type=offline
Paste the authorization code here: 4/1AVMBsJgQouTmpyXemky6NFTQH2glbdA2kNcuQBUJzg2Fvn3ijhq4eTchd-g


In [ ]:
from datetime import datetime, timedelta
import pytz

def is_conflict(service, start_time, end_time):
    events_result = service.events().list(
        calendarId='primary',
        timeMin=start_time,
        timeMax=end_time,
        singleEvents=True,
        orderBy='startTime'
    ).execute()
    return len(events_result.get('items', [])) > 0

def view_calendar(service, max_results=5):
    now = datetime.now(pytz.timezone('Asia/Riyadh'))
    end = now + timedelta(days=1)
    events = service.events().list(calendarId='primary', timeMin=now.isoformat(), timeMax=end.isoformat(), singleEvents=True, orderBy='startTime').execute().get('items', [])
    calendar_output = []
    for event in events:
        start = event['start'].get('dateTime', event['start'].get('date'))
        summary = event.get('summary', 'No Title')
        calendar_output.append(f"- {start} — {summary}")
    return "\n".join(calendar_output) if calendar_output else "No events scheduled today."

def add_event(service, summary, start_time, end_time):
    event = {
        'summary': summary,
        'start': {
            'dateTime': start_time,
            'timeZone': 'Asia/Riyadh',
        },
        'end': {
            'dateTime': end_time,
            'timeZone': 'Asia/Riyadh',
        },
    }
    event = service.events().insert(calendarId='primary', body=event).execute()
    print(f"✅ Event '{summary}' added to calendar.")

def delete_event(service, title):
    now = datetime.utcnow().isoformat() + 'Z'
    end_of_day = (datetime.utcnow().replace(hour=23, minute=59, second=59)).isoformat() + 'Z'
    events_result = service.events().list(
        calendarId='primary',
        timeMin=now,
        timeMax=end_of_day,
        singleEvents=True,
        orderBy='startTime'
    ).execute()
    events = events_result.get('items', [])

    for event in events:
        if title.lower() in event['summary'].lower():
            service.events().delete(calendarId='primary', eventId=event['id']).execute()
            print(f"🗑️ Deleted event: {event['summary']}")
            return

    print(f"⚠️ No matching event found for deletion: {title}")


In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime

scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("service_account.json", scope)
gs = gspread.authorize(creds)
sheet = gs.open("Legal Agent Chat Log").sheet1

def log_to_sheet(user_id, question, answer):
    sheet.append_row([user_id, question, answer, datetime.now().isoformat()])

In [ ]:
import smtplib
from email.mime.text import MIMEText

GMAIL = 'ahmadfaresmzayek@gmail.com'
GMAIL_PASS = 'jokw thfg hbko zwze'

def send_alert_email(question):
    msg = MIMEText(f"A legal question was just asked:\n\n{question}")
    msg['Subject'] = "🧠 Legal AI Agent Notification"
    msg['From'] = GMAIL
    msg['To'] = GMAIL

    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as s:
        s.login(GMAIL, GMAIL_PASS)
        s.send_message(msg)

In [ ]:
from dateutil.parser import isoparse
import dateparser
import re
from difflib import SequenceMatcher
import pytz

user_id = "user-001"
while True:
    question = input("Ask your legal question (or type 'quit agent' to exit): ")
    if question.lower().strip() == "quit agent":
        print("👋 Exiting assistant.")
        break

    calendar_today = view_calendar(calendar_service)

    prompt_context = f"""
User calendar for today:
{calendar_today}

User question:
{question}
"""

    completion = client.chat.completions.create(
        model="deepseek/deepseek-r1-0528-qwen3-8b:free",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt_context}
        ]
    )

    response = completion.choices[0].message.content
    print("\n🤖 Response:\n", response)

    # 🔍 Intent Detection
    if any(k in question.lower() for k in ["add", "schedule", "book"]):
        match = re.search(r"(?:meeting|event|appointment) (?:with|titled|called|named)? ['\"]?(.+?)['\"]? (?:at|on)? (.+)", question, re.IGNORECASE)
        if match:
            title = match.group(1).strip().strip('"').strip("'")
            time = dateparser.parse(match.group(2))
            if time:
                start = time.replace(second=0).isoformat()
                end = (time + timedelta(hours=1)).replace(second=0).isoformat()
                add_event(calendar_service, title, start, end)

    elif any(k in question.lower() for k in ["cancel", "delete"]):
        match = re.search(
            r'(?:cancel|delete)\s+(?:the\s+)?(?:event|meeting|appointment)?\s*(?:named|titled|called)?\s*[\'\"]?(.+?)[\'\"]?(?:\s+at\s+(.+?))?(?:\s+today)?[\.\s]*$',
            question, re.IGNORECASE
        )
        if match:
            title = match.group(1).strip().strip('"').strip("'")
            time_str = match.group(2).strip() if match.group(2) else None
            target_time = dateparser.parse(time_str) if time_str else None

            now = datetime.utcnow().isoformat() + 'Z'
            end_of_day = (datetime.utcnow().replace(hour=23, minute=59, second=59)).isoformat() + 'Z'
            events_result = calendar_service.events().list(
                calendarId='primary',
                timeMin=now,
                timeMax=end_of_day,
                singleEvents=True,
                orderBy='startTime'
            ).execute()
            events = events_result.get('items', [])

            best_match = None
            best_ratio = 0.0
            for event in events:
                summary = event['summary'].strip().lower()
                start_time = event['start'].get('dateTime', '')
                event_time = isoparse(start_time) if start_time else None

                ratio = SequenceMatcher(None, summary, title.lower()).ratio()
                time_diff = None
                if event_time and target_time:
                    event_time_utc = event_time.astimezone(pytz.utc)
                    target_time_utc = target_time.astimezone(pytz.utc)
                    time_diff = abs((event_time_utc - target_time_utc).total_seconds())

                if ratio > best_ratio and (time_diff is None or time_diff <= 1800):
                    best_match = (event, event_time)
                    best_ratio = ratio

            if best_match and best_ratio >= 0.7:
                confirm = input(f"⚠️ Confirm deletion of '{best_match[0]['summary']}' scheduled at {best_match[1].strftime('%H:%M')}? (yes/no): ")
                if confirm.lower() == "yes":
                    calendar_service.events().delete(calendarId='primary', eventId=best_match[0]['id']).execute()
                    print(f"🗑️ Deleted event: {best_match[0]['summary']}")
                    response = f"""
The event **{best_match[0]['summary']}** scheduled for **{best_match[1].strftime('%Y-%m-%d %H:%M')}** has been deleted from your calendar.

**Logging Details:**
- Date: {best_match[1].strftime('%Y-%m-%d')}
- Time: {best_match[1].strftime('%H:%M')}
- Title: {best_match[0]['summary']}
- Action: Deletion
"""
                else:
                    print("❌ Deletion cancelled.")
                    response = f"Deletion of event '{best_match[0]['summary']}' was cancelled by the user."
            else:
                print(f"⚠️ No matching event found for deletion: {title}")
                response = f"⚠️ No matching calendar event found for deletion: \"{title}\" today at {target_time.strftime('%H:%M') if target_time else 'unspecified time'}."

    log_to_sheet(user_id, question, response)
    send_alert_email(question)


🤖 Response:
 I apologize, as your Legal Assistant. After reviewing the provided information and the user's query regarding a scheduled event for 4th July 2025, I must inform you that I cannot retrieve or access future calendar events. My system is currently operational under the constraints of the Labor Law and policy framework as per your guidelines. 

Please refer to your personal calendar device system for future dates.

However, I am programmed according to Labor Law regulations which may impose time-sensitive obligations. Are you concerned about meeting any compliance deadlines, or perhaps there is an upcoming scheduled consultation regarding employment agreements or work organization regulations? I am ready to assist with such legitimate inquiries.

🤖 Response:
 I cannot view your future calendar events. For July 4, 2025, please save this calendar query for a reminder. 

If you have other calendars or documents, please share them so I can help you log the needed information.

🤖 